In [1]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

import time
import datetime

import extract_features as ef


### Чтение файлов с наборами данных

Считайте таблицу с признаками из файла features.csv 

In [2]:
train_features = pd.read_csv('./features.csv', index_col='match_id')
test_features = pd.read_csv('./features_test.csv', index_col='match_id')

Удалиv признаки, связанные с итогами матча ( помечены в описании данных как отсутствующие в тестовой выборке).

In [3]:
delete_columns_features = [x for x in set(train_features.columns) - set(test_features.columns)]  

print delete_columns_features

['tower_status_dire', 'barracks_status_dire', 'barracks_status_radiant', 'tower_status_radiant', 'duration', 'radiant_win']


In [4]:
X_train = train_features.drop(delete_columns_features, axis = 1)
X_test = test_features

y_train = train_features.radiant_win

Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [9]:
for i in X_train.columns:
    if X_train[i].count() < X_train.shape[0]:
        print i, ', nan values = ', X_train.shape[0] - X_train[i].count()

first_blood_time , nan values =  19553
first_blood_team , nan values =  19553
first_blood_player1 , nan values =  19553
first_blood_player2 , nan values =  43987
radiant_bottle_time , nan values =  15691
radiant_courier_time , nan values =  692
radiant_flying_courier_time , nan values =  27479
radiant_first_ward_time , nan values =  1836
dire_bottle_time , nan values =  16143
dire_courier_time , nan values =  676
dire_flying_courier_time , nan values =  26098
dire_first_ward_time , nan values =  1826


#### Объяснение пропусков

first_blood_time и firstblood_team - одинаковое количество пропусков. Возможно, данные бои не состоялись как "игра" - то есть "первая кровь" и не была пролита.

radiant_courier_time - курьер не был куплен


###### Замените пропуски на нули с помощью функции fillna().

In [10]:
X_train = X_train.fillna(0)

##### Какая переменная целевая?

radiant_win - т.к. решаем задачу о победе команды

### 1. Градиентный бустинг


In [11]:
kf = KFold(len(X_train), n_folds=5, shuffle=True)

X_train_for_clf = np.array(X_train)
y_train_for_clf = np.array(y_train)

In [12]:
# Предсказания на кросс-валидации

for n_est in [10,20,30]:
    start_time = datetime.datetime.now()

    print 'n_estimators = ', n_est
    print "score in kf: "
    for training, validation in kf:
        clf = GradientBoostingClassifier(n_estimators=n_est)
        clf.fit(X_train_for_clf[training], y_train_for_clf[training])
        prediction = clf.predict(X_train_for_clf[validation])

        score = roc_auc_score(y_train_for_clf[validation], prediction)
        print score
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print '_'*10 
    

n_estimators =  10
score in kf: 
0.610114253002
0.605779728549
0.605560947567
0.612608326334
0.603095652975
Time elapsed: 0:03:45.223000
__________
n_estimators =  20
score in kf: 
0.626244412001
0.625553302743
0.621049381765
0.629332877339
0.621749810808
Time elapsed: 0:07:46.446000
__________
n_estimators =  30
score in kf: 
0.634200588764
0.632156452262
0.629189282907
0.636543187197
0.628760907135
Time elapsed: 0:23:16.942000
__________


### 2. Логистическая регрессия


Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, scale

In [13]:
X_train_for_clf = StandardScaler().fit_transform(X_train_for_clf)

In [15]:
for c_value in np.arange (0.005,0.01,0.001):
    print "c_value = %3f, score in kf: " %c_value
    for training, validation in kf:
        clf = LogisticRegression(penalty= 'l2', C = c_value)
        clf.fit(X_train_for_clf[training], y_train_for_clf[training])
        
        prediction = clf.predict(X_train_for_clf[validation])
        score = accuracy_score(y_train_for_clf[validation], prediction)
        print score


c_value = 0.005000, score in kf: 
0.656536048545
0.654941890363
0.652576365319
0.660187185025
0.652936336522
c_value = 0.006000, score in kf: 
0.65663889746
0.654993314821
0.65216496966
0.660752854057
0.652833487607
c_value = 0.007000, score in kf: 
0.656484624087
0.654684768076
0.652010696287
0.661009976345
0.652987760979
c_value = 0.008000, score in kf: 
0.656484624087
0.654479070246
0.652216394117
0.66111282526
0.653193458809
c_value = 0.009000, score in kf: 
0.656484624087
0.654427645788
0.652422091947
0.66111282526
0.653193458809


Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [14]:
uncategorial_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']

In [15]:
for item in uncategorial_features:
    del X_train[item]

In [16]:
X_train_for_clf = np.array(X_train)
y_train_for_clf = np.array(y_train)

X_train_for_clf = StandardScaler().fit_transform(X_train_for_clf)

In [19]:
for c_value in np.arange (0.005,0.01,0.001):
    print "c_value = %3f, score in kf: " %c_value
    for training, validation in kf:
        clf = LogisticRegression(penalty= 'l2', C = c_value)
        clf.fit(X_train_for_clf[training], y_train_for_clf[training])
        
        prediction = clf.predict(X_train_for_clf[validation])
        score = accuracy_score(y_train_for_clf[validation], prediction)
        print score


c_value = 0.005000, score in kf: 
0.656278926257
0.654067674586
0.651702149542
0.6607014296
0.653193458809
c_value = 0.006000, score in kf: 
0.6562275018
0.654016250129
0.651599300627
0.660392882855
0.653142034352
c_value = 0.007000, score in kf: 
0.656381775172
0.653964825671
0.651496451712
0.660392882855
0.653193458809
c_value = 0.008000, score in kf: 
0.656073228427
0.654119099044
0.651599300627
0.660444307313
0.653502005554
c_value = 0.009000, score in kf: 
0.656278926257
0.654119099044
0.651856422915
0.660444307313
0.653502005554


Удаление этих признаков, не повлияло на качество обучения выборки. Возможно, данные признаки не являются релевантными для данного набора данных.


На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [22]:
num_players = train_features.r1_hero.unique().max()

print num_players

112


Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [19]:
X_pick = np.zeros((train_features.shape[0], num_players))

for i, match_id in enumerate(X_train.index):
    for p in xrange(5):
        X_pick[i, train_features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, train_features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [20]:
for i in xrange(num_players):
    X_train['h_%d' % i] = X_pick[:,i]

Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [21]:
X_train_for_clf = np.array(X_train)
X_train_for_clf =  StandardScaler().fit_transform(X_train_for_clf)

In [ ]:
kf = KFold(len(X_train_for_clf), n_folds=5, shuffle=True)

In [26]:
for c_value in np.arange (0.005,0.01,0.001):
    print "c_value = %3f, score in kf: " %c_value
    for training, validation in kf:
        clf = LogisticRegression(penalty= 'l2', C = c_value, random_state=240)
        clf.fit(X_train_for_clf[training], y_train_for_clf[training])
        
        prediction = clf.predict(X_train_for_clf[validation])
        score = roc_auc_score(y_train_for_clf[validation], prediction)
        print score


c_value = 0.005000, score in kf: 
0.681171380706
0.684096874551
0.677690552611
0.680696255871
0.687083302163
c_value = 0.006000, score in kf: 
0.681117910489
0.684184945213
0.677732906956
0.680839740529
0.687096516988
c_value = 0.007000, score in kf: 
0.681373439333
0.68375063981
0.677683244659
0.680588786769
0.686890607134
c_value = 0.008000, score in kf: 
0.681522027743
0.683594299821
0.677942518068
0.680189918601
0.686724611771
c_value = 0.009000, score in kf: 
0.681216969429
0.683544672029
0.677935210117
0.680037574445
0.686359313966


Качество предсказания улучшилось. Объяснить это можно появлением нового, более информативного признака.

### Лучшее предсказание на логичтиечской регрессии



In [143]:
kf = KFold(len(X_train_for_clf), n_folds=5, shuffle=True)

X_train_for_clf = StandardScaler().fit_transform(X_train)
clf = LogisticRegression(penalty= 'l2', C = 0.01, random_state=240)
score = cross_val_score(clf, X_train_for_clf, y_train, cv = kf, scoring='roc_auc').mean()
print score

0.751900517466


In [23]:
X_test = pd.read_csv('./features_test.csv', index_col='match_id')
X_test = X_test.fillna(value = 0)

### Добавляем новые признаки в тест

In [24]:
X_pick = np.zeros((X_test.shape[0], num_players))

for i, match_id in enumerate(X_test.index):
    for p in xrange(5):
        X_pick[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
num_players  = X_test.r1_hero.unique().max()

for i in xrange(num_players):
    X_test['h_%d' % i] = X_pick[:,i]
    
# Удаляем некатегориальные признаки

for item in uncategorial_features:
    del X_test[item]

In [27]:
X_test_for_clf = StandardScaler().fit_transform(X_test)

In [25]:
print X_train.shape, X_test.shape

(97230, 203) (17177, 203)


In [28]:
clf = LogisticRegression(penalty= 'l2', C = 0.01, random_state=240)
clf.fit(X_train_for_clf, y_train)

prediction_win =clf.predict_proba(X_test_for_clf)

In [31]:
print prediction_win.max(), prediction_win.min()

0.996328715925 0.00367128407457


In [153]:
prediction_win =clf.predict_proba(X_test_for_clf)
Y_pred = clf.predict(X_test_for_clf)

In [178]:
X_test['radiant_win'] = prediction_win[:,1]

In [201]:
d = pd.DataFrame({'radiant_win':X_test.radiant_win})

In [200]:
d.to_csv('predictions.csv')